## Recieving Stats
Aggregate individual recieving stats by game from the passing csv into player stats per year for each team. Meant to be run after the rushing notebook.

In [26]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None

In [27]:
#Get recieving df
recieving = pd.read_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\recieving.csv')
recieving.head()

,year,game_id,team,name,long,receptions,touchdowns,yards
0,2009,2009091000,TEN,C.Johnson,11,1,0,11
1,2009,2009091000,TEN,L.White,5,1,0,5
2,2009,2009091000,TEN,J.Gage,16,7,1,78
3,2009,2009091000,TEN,B.Scaife,20,5,0,48
4,2009,2009091000,TEN,K.Britt,57,4,0,85


In [28]:
#Get player stats to write to
player_stats = pd.read_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\players.csv')
player_stats.head()

,year,team,pass_to_run,completion_perc,pass_len,interceptions_turnover,pass_touchdowns,rush_touchdowns,running_back_avg
0,2009,ARI,0.62,0.66,10.70,18,28,15,4.51
1,2009,ATL,0.56,0.58,11.14,17,26,15,4.89
2,2009,BAL,0.52,0.63,11.33,13,21,22,5.27
3,2009,BUF,0.51,0.58,10.89,19,17,6,4.46
4,2009,CAR,0.47,0.57,11.63,20,16,18,5.13


In [29]:
#Aggregate recieving stats, filter out recievers with less than 10 catches
r_agg = recieving.groupby(['year','team','name'])['receptions','touchdowns','yards'].sum().reset_index()
r_agg['yards_per_catch'] = r_agg['yards'].divide(r_agg['receptions']).round(2)
r_agg = r_agg[r_agg['receptions'] >= 10]
r_agg.head()

,year,team,name,receptions,touchdowns,yards,yards_per_catch
1,2009,ARI,A.Boldin,85,5,1029,12.11
2,2009,ARI,B.Patrick,12,2,146,12.17
3,2009,ARI,C.Wells,12,0,143,11.92
5,2009,ARI,E.Doucet,17,1,214,12.59
6,2009,ARI,J.Urban,18,0,186,10.33


In [30]:
#Add max touchdown reciever, and max yard per catch reciever
r_max = r_agg.groupby(['year','team'])['touchdowns','yards_per_catch','yards'].max().reset_index()
player_stats2 = player_stats.merge(r_max,on=['year','team'])
player_stats3 = player_stats2.rename(columns={'touchdowns': 'rec_max_touchdowns','yards_per_catch':'max_yards_per_catch','yards':'rec_yards_max'})
player_stats3.head()

,year,team,pass_to_run,completion_perc,pass_len,interceptions_turnover,pass_touchdowns,rush_touchdowns,running_back_avg,rec_max_touchdowns,max_yards_per_catch,rec_yards_max
0,2009,ARI,0.62,0.66,10.70,18,28,15,4.51,13,12.95,1092
1,2009,ATL,0.56,0.58,11.14,17,26,15,4.89,11,13.56,1153
2,2009,BAL,0.52,0.63,11.33,13,21,22,5.27,7,14.12,1028
3,2009,BUF,0.51,0.58,10.89,19,17,6,4.46,7,15.07,829
4,2009,CAR,0.47,0.57,11.63,20,16,18,5.13,7,20.17,982


In [55]:
#Add spread between reciever receptions, number of threat feature
rec_count = r_agg[r_agg['receptions'] >= 20].groupby(['year','team'])['receptions'].std().round(2).to_frame().reset_index()
player_stats4 = player_stats3.merge(rec_count, on=['year','team'], how='left')
player_stats5 = player_stats4.rename(columns={'receptions':'rec_spread'})
player_stats5.head()

,year,team,pass_to_run,completion_perc,pass_len,interceptions_turnover,pass_touchdowns,rush_touchdowns,running_back_avg,rec_max_touchdowns,max_yards_per_catch,rec_yards_max,rec_spread
0,2009,ARI,0.62,0.66,10.70,18,28,15,4.51,13,12.95,1092,19.39
1,2009,ATL,0.56,0.58,11.14,17,26,15,4.89,11,13.56,1153,26.70
2,2009,BAL,0.52,0.63,11.33,13,21,22,5.27,7,14.12,1028,23.09
3,2009,BUF,0.51,0.58,10.89,19,17,6,4.46,7,15.07,829,12.14
4,2009,CAR,0.47,0.57,11.63,20,16,18,5.13,7,20.17,982,18.27


In [61]:
#Save to csv
player_stats5.to_csv('C:\\Users\\Louie\\Repositories\\Springboard\\Capstone 1 - NFL Matchup Predictions\\Data Wrangling\\Data\\players.csv',index=False)